<a href="https://colab.research.google.com/github/bacchus00/limpieza-aportes-servel/blob/main/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
from google.colab import drive
drive.mount('/content/drive')

work_dir = '/content/drive/MyDrive/Tesis/Trabajo Dirigido/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
!pip install unidecode
!pip install swifter
!pip install chile_rut

In [128]:
import pandas as pd
import re
import numpy as np
import hashlib
import swifter
import chile_rut

from unidecode import unidecode

# Importing

In [129]:
# importing data
raw_data = pd.read_csv(work_dir + 'data/aportes-raw.csv')

<ipython-input-129-f4dde9863089>:2: DtypeWarning: Columns (1,2,4,5,9,11,12,17,18,23,24,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(work_dir + 'data/aportes-raw.csv')


In [130]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488432 entries, 0 to 488431
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Archivo                    488432 non-null  object 
 1   RUTAportante               344231 non-null  object 
 2   DVAportante                228466 non-null  object 
 3   Aportante                  442385 non-null  object 
 4   RUTCandidato               481383 non-null  object 
 5   DVCandidato                397180 non-null  object 
 6   Candidato                  486498 non-null  object 
 7   Afiliacion                 480705 non-null  object 
 8   RUTPartido                 1705 non-null    float64
 9   DVPartido                  1705 non-null    object 
 10  Partido                    438152 non-null  object 
 11  Pacto                      225156 non-null  object 
 12  Subpacto                   147134 non-null  object 
 13  Fecha                      42

In [131]:
# general cleansing
df = raw_data.copy()

df = df.applymap(lambda x: unidecode(' '.join(x.split()).title()) if type(x)==str else x)
df['Archivo'] = df['Archivo'].map(lambda x: x.lower())

df = df.replace({'-': np.nan, '':np.nan,})

df = df.drop('TipoPlanilla', axis=1)

# Rows With Errors

In [132]:
df[df['Eleccion'].isna() | (df['Eleccion']=='Ingresos Rechazados')]

,Archivo,RUTAportante,DVAportante,Aportante,RUTCandidato,DVCandidato,Candidato,Afiliacion,RUTPartido,DVPartido,...,DescripcionTipoDocumento,GlosaDocumento,Eleccion,Comuna,Region,Distrito,Circunscripcion,CircunscripcionProvincial,CircunscripcionSenatorial,TerritorioElectoral
29960,2004-alcalde-consejal-ingresos-partidos.xls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112544,2013-presidente-senador-diputado-core-ingresos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112545,2013-presidente-senador-diputado-core-ingresos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112546,2013-presidente-senador-diputado-core-ingresos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Ingresos Rechazados,NaN,*,NaN,NaN,NaN,NaN,NaN


In [133]:
drop_rows = df.index[
    df['Eleccion'].isna() |
    (df['Eleccion']=='Ingresos Rechazados')
]

df = df.drop(drop_rows)

# Monto

In [134]:
# replacing 0 for nans
df['Monto'] = df['Monto'].replace({0: np.nan})

In [135]:
df['Monto'].describe()

count    4.801260e+05
mean     9.519577e+05
std      1.490500e+07
min      1.000000e+00
25%      2.500000e+04
50%      1.000000e+05
75%      3.500000e+05
max      3.578743e+09
Name: Monto, dtype: float64

# Tipo Beneficiario

In [136]:
# new column 'TipoBeneficiario' from filename
df['TipoBeneficiario'] = df['Archivo'].map(lambda x: 'Partido' if 'partido' in x else 'Candidato')

In [137]:
df['TipoBeneficiario'].value_counts(dropna=False)

Candidato    484074
Partido        4354
Name: TipoBeneficiario, dtype: int64

# Eleccion

In [138]:
# new column for election year from filename year
df['AnioEleccion'] = df['Archivo'].map(lambda x: x[:4])

In [139]:
df['AnioEleccion'].value_counts(dropna=False)

2021    214206
2016    111536
2017     46715
2008     35626
2004     29960
2012     27655
2013      8909
2005      5708
2009      4840
2020      3273
Name: AnioEleccion, dtype: int64

In [140]:
# renaming elections from filename information
df['Eleccion'] = df['Eleccion'].replace(to_replace={'Segunda Votacion Presidente':'Presidente'})

In [141]:
df['Eleccion'].value_counts(dropna=False)

Concejal                      230719
Alcalde                        91313
Diputado                       63644
Presidente                     29546
Convencional Constituyente     27856
Consejero Regional             22680
Senador                        16964
Gobernador Regional             5706
Name: Eleccion, dtype: int64

In [142]:
# new column for election typefrom filename year
df['TipoEleccion'] = df['Archivo'].map(lambda x: 'Primarias' if 'primaria' in x else 'Regular')
df['TipoEleccion'] = df.swifter.apply(lambda x: 'Segunda' if '2da' in x['Archivo'] else x['TipoEleccion'], axis=1)

Pandas Apply:   0%|          | 0/488428 [00:00<?, ?it/s]

In [143]:
df['TipoEleccion'].value_counts(dropna=False)

Regular      467317
Segunda       11551
Primarias      9560
Name: TipoEleccion, dtype: int64

In [144]:
df[['AnioEleccion','Eleccion','TipoEleccion']].value_counts(dropna=False).reset_index().sort_values(['AnioEleccion','Eleccion','TipoEleccion'])

,AnioEleccion,Eleccion,TipoEleccion,0
14,2004,Alcalde,Regular,7533
7,2004,Concejal,Regular,22427
17,2005,Diputado,Regular,4716
39,2005,Presidente,Regular,75
41,2005,Presidente,Segunda,61
29,2005,Senador,Regular,856
15,2008,Alcalde,Regular,7529
5,2008,Concejal,Regular,28097
19,2009,Diputado,Regular,3996
36,2009,Presidente,Regular,140


# Territorio Electoral

In [145]:
pd.set_option('display.max_rows', None)

In [146]:
territorios = [
  'TerritorioElectoral',
  'Circunscripcion',
  'Region',
  'CircunscripcionSenatorial',
  'Distrito',
  'CircunscripcionProvincial',
  'Comuna',

]

In [147]:
replace_territorio = {
    'Nacional':np.nan,
    'Pais':np.nan,
    'Circunscripcion Senatorial':np.nan,
    'Region':np.nan,
    'Partido': np.nan,
    'Partido Politico': np.nan
}

df['TerritorioElectoral'] = df['TerritorioElectoral'].replace(replace_territorio)

In [148]:
replace_region = {
    'Nacional': np.nan,
    'Partido': np.nan,
    'Partido Politico': np.nan
}

df['Region'] = df['Region'].replace(replace_region)

In [149]:
# valid election territories
territorios_validos = [
  'Region',
  'CircunscripcionSenatorial',
  'Distrito',
  'CircunscripcionProvincial',
  'Comuna',
]

# fills election territory columns according to the election
def replace_territorio(x): #refactorizable
  region = np.nan
  cir_sen = np.nan
  distrito = np.nan
  cir_prov = np.nan
  comuna = np.nan

  if x['Eleccion']=='Gobernador Regional':
    if x['Region']==x['Region']:
      region = x['Region']
    else:
      region = x['TerritorioElectoral']

  elif x['Eleccion']=='Consejero Regional':
    if x['CircunscripcionProvincial']==x['CircunscripcionProvincial']:
      cir_prov = x['CircunscripcionProvincial']
    elif x['Circunscripcion']==x['Circunscripcion']:
      cir_prov = x['Circunscripcion']
    else:
      cir_prov = x['TerritorioElectoral']
    region = x['Region']

  elif x['Eleccion']=='Senador':
    if x['CircunscripcionSenatorial']==x['CircunscripcionSenatorial']:
      cir_sen = x['CircunscripcionSenatorial']
    elif x['Circunscripcion']==x['Circunscripcion']:
      cir_sen = x['Circunscripcion']
    else:
      cir_sen = x['TerritorioElectoral']
    region = x['Region']

  elif x['Eleccion']=='Diputado' or x['Eleccion']=='Convencional Constituyente':
    if x['Distrito']==x['Distrito']:
      distrito = x['Distrito']
    else:
      distrito = x['TerritorioElectoral']
    region = x['Region']

  elif x['Eleccion']=='Alcalde' or x['Eleccion']=='Concejal':
    if x['Comuna']==x['Comuna']:
      comuna = x['Comuna']
    else:
      comuna = x['TerritorioElectoral']
    region = x['Region']

  return pd.Series([region, cir_sen, distrito, cir_prov, comuna], index=territorios_validos)

df[territorios_validos] = df.swifter.apply(replace_territorio, axis=1)

Pandas Apply:   0%|          | 0/488428 [00:00<?, ?it/s]

In [150]:
# importing electoral division
div_parl = pd.read_csv(work_dir+'data/'+'division-electoral-parlamentaria-chile-1989-2018.csv')

# creating comuna-ditrito and comuna cicunscripcion-electoral dictionaries
comuna_distrito = pd.Series(div_parl['Distrito'].values,index=div_parl['Comuna']).to_dict()
comuna_cir_sen = pd.Series(div_parl['Circunscripcion'].values,index=div_parl['Comuna']).to_dict()

#replacing territorie values
region_replace = {
    '13': 'Region Metropolitana De Santiago',
    '6': "Region Del Libertador General Bernardo O'Higgins",
    '9': 'Region De La Araucania',
    '16': 'Region De Nuble',
    '11': 'Region De Aysen Del General Carlos Ibanez Del Campo',
    '7': 'Region Del Maule',
    '3': 'Region De Atacama',
    '2': 'Region De Antofagasta',
    '5': 'Region De Valparaiso',
    '1': 'Region De Tarapaca',
    '8': 'Region Del Biobio',
    '14': 'Region De Los Rios',
    '4': 'Region De Coquimbo',
    '15': 'Region De Arica Y Parinacota',
    '10': 'Region De Los Lagos',
    '12': 'Region De Magallanes Y De La Antartica Chilena',
    'Partido Politico': np.nan,
    'Region': np.nan

}

cir_sen_replace = {
    'Circunscripcion Senatorial': np.nan,
    'CircunscripcionSenatorial': np.nan,
    'Aisen':'Aysen',
}

distrito_replace = {
    'Distrito': np.nan,
    'Aisen':'Aysen',
    'Llay-Llay': 'Llaillay',
    'O Higgins':"O'Higgins",
}

cir_prov_replace = {
    'Circunscripcion Provincial': np.nan,
    'Bio Bio': 'Biobio',
    'Cachapoal I': 'Cachapoal 1',
    'Cachapoal Ii': 'Cachapoal 2',
    'Cautin I': 'Cautin 1',
    'Cautin Ii': 'Cautin 2',
    'Concepcion I': 'Concepcion 1',
    'Concepcion Ii': 'Concepcion 2',
    'Concepcion Iii': 'Concepcion 3',
    'Santiago I': 'Santiago 1',
    'Santiago Ii': 'Santiago 2',
    'Santiago Iii': 'Santiago 3',
    'Santiago Iv': 'Santiago 4',
    'Santiago V': 'Santiago 5',
    'Santiago Vi': 'Santiago 6',
    'Valparaiso I': 'Valparaiso 1',
    'Valparaiso Ii': 'Valparaiso 2',
}

comuna_replace = {
    'Alto Bio Bio': 'Alto Biobio',
    'O Higgins': "O'Higgins",
    'O`Higgins': "O'Higgins",
    'Partido Politico': np.nan,
}

df['Region'] = df['Region'].map(lambda x: str(x) if x==x else x)
df['Region'] = df['Region'].replace(region_replace)

df['CircunscripcionSenatorial'] = df['CircunscripcionSenatorial'].map(lambda x: str(x).replace('.0','').replace('Circunscripcion Senatorial Ndeg','') if x==x else x)
df['CircunscripcionSenatorial'] = df['CircunscripcionSenatorial'].replace(cir_sen_replace)
df['CircunscripcionSenatorial'] = df['CircunscripcionSenatorial'].map(lambda x: str(comuna_cir_sen[x]) if x in comuna_cir_sen.keys() else x)

df['Distrito'] = df['Distrito'].map(lambda x: str(x).replace('.0','').replace('Distrito Ndeg','').replace('Distrito No ','') if x==x else x)
df['Distrito'] = df['Distrito'].replace(distrito_replace)
df['Distrito'] = df['Distrito'].map(lambda x: str(comuna_distrito[x]) if x in comuna_distrito.keys() else x)

df['CircunscripcionProvincial'] = df['CircunscripcionProvincial'].replace(cir_prov_replace)

df['Region'] = df['Region'].replace(region_replace)

df['Comuna'] = df['Comuna'].replace(comuna_replace)

In [151]:
df['Region'].value_counts(dropna=False)

NaN                                                    137212
Region Metropolitana De Santiago                        96697
Region Del Biobio                                       37701
Region De Valparaiso                                    35500
Region Del Maule                                        31206
Region Del Libertador General Bernardo O'Higgins        26802
Region De La Araucania                                  25206
Region De Los Lagos                                     20216
Region De Tarapaca                                      16087
Region De Coquimbo                                      13426
Region De Los Rios                                       9358
Region De Atacama                                        8360
Region De Antofagasta                                    7996
Region De Nuble                                          7209
Region De Magallanes Y De La Antartica Chilena           5480
Region De Aysen Del General Carlos Ibanez Del Campo      5128
Region D

In [152]:
df['CircunscripcionSenatorial'].value_counts(dropna=False)

NaN    471863
1        6130
10       2092
7        1401
15       1313
3         942
6         730
8         719
12        629
5         604
13        430
18        375
16        345
14        252
9         121
17        111
4         101
11         97
2          95
19         78
Name: CircunscripcionSenatorial, dtype: int64

In [153]:
df['Distrito'].value_counts(dropna=False)

NaN    398584
10       5006
11       3877
7        3459
21       3381
23       3290
20       3195
8        3133
28       3072
9        2819
17       2516
12       2487
6        2454
22       2331
13       2314
14       2275
19       2263
16       2252
15       2198
4        2046
2        2045
5        2007
26       1971
18       1820
24       1558
1        1501
44       1370
25       1342
3        1326
27       1286
34       1224
37       1173
57       1080
31       1058
40        965
50        939
30        939
35        887
33        872
53        776
60        708
42        658
55        633
43        605
36        602
47        546
41        523
59        521
49        462
56        456
48        448
32        441
58        427
39        418
54        400
29        328
46        262
51        257
45        233
38        232
52        177
Name: Distrito, dtype: int64

In [154]:
df['CircunscripcionProvincial'].value_counts(dropna=False)

NaN                  466054
Iquique                 759
Valdivia                714
Arica                   705
Concepcion 2            678
Elqui                   663
Antofagasta             653
Talca                   616
Cautin 1                598
Santiago 4              576
Santiago 5              551
Cachapoal 2             534
Valparaiso 1            526
Cautin 2                502
Concepcion 1            501
Valparaiso 2            501
Santiago 2              474
Marga Marga             463
Concepcion 3            459
Curico                  458
Santiago 6              448
Copiapo                 446
Santiago 1              443
Biobio                  428
Llanquihue              427
Magallanes              404
Limari                  397
Coyhaique               394
Osorno                  390
Linares                 387
El Loa                  382
Diguillin               379
Santiago 3              359
Maipo                   336
Choapa                  328
Colchagua           

In [155]:
df['Comuna'].value_counts(dropna=False)

NaN                           167385
Iquique                         5473
Providencia                     2998
Valparaiso                      2962
Maipu                           2817
Vina Del Mar                    2800
Santiago                        2694
Puerto Montt                    2689
Concepcion                      2664
Talca                           2572
Penalolen                       2439
Nunoa                           2366
Antofagasta                     2213
La Pintana                      2188
Temuco                          2142
La Florida                      2129
San Bernardo                    2128
Rancagua                        2102
Las Condes                      2074
Arica                           2062
Talcahuano                      2062
Pudahuel                        2056
Talagante                       2036
Conchali                        1997
Puente Alto                     1922
Hualpen                         1917
Copiapo                         1891
P

In [156]:
# new columns TerritorioElectoral name of smallest territory column corresponding to election
eleccion_territorio_replace = {
  'Presidente': 'Pais',
  'Gobernador Regional': 'Region',
  'Consejero Regional': 'CircunscripcionProvincial',
  'Senador': 'CircunscripcionSenatorial',
  'Diputado': 'Distrito',
  'Convencional Constituyente': 'Distrito',
  'Alcalde': 'Comuna',
  'Concejal': 'Comuna',
}

df['TerritorioElectoral'] = df['Eleccion'].map(lambda x: eleccion_territorio_replace[x])

In [157]:
df['TerritorioElectoral'].value_counts(dropna=False)

Comuna                       322032
Distrito                      91500
Pais                          29546
CircunscripcionProvincial     22680
CircunscripcionSenatorial     16964
Region                         5706
Name: TerritorioElectoral, dtype: int64

# Afiliacion

In [158]:
# correcting afiliation status of candidates

archivo_afiliacion = {
  '2005-presidente-2da-ingresos-candidatos.xls': 'Afiliado',
  '2009-presidente-2da-ingresos-candidatos.xls': 'Afiliado',
  '2017-presidente-2da-ingresos-candidatos.xlsx': 'Independiente En Pacto'
}

candidato_prim_afiliacion = {
  'Daniel Jadue Jadue': 'Afiliado',
  'Gabriel Boric Font': 'Afiliado',
  'Ignacio Briones Rojas': 'Afiliado',
  'Joaquin Lavin Infante': 'Afiliado',
  'Mario Desbordes Jimenez': 'Afiliado',
  'Sebastian Sichel Ramirez': 'Independiente Asociado',
}

afiliacion_replace = {
  'Independiente En Pacto': 'Independiente Asociado',
  'Independiente En Subpacto': 'Independiente Asociado',
  'Indpendiente De Pacto': 'Independiente Asociado',
  'Independiente Asociado': 'Independiente Asociado',
  'Candidatura Independiente': 'Independiente No Asociado',
  'Independiente Fuera De Pacto': 'Independiente No Asociado',
}

df['Afiliacion'] = df.apply(lambda x: archivo_afiliacion[x['Archivo']] if x['Archivo'] in archivo_afiliacion else x['Afiliacion'], axis=1)
df['Afiliacion'] = df.apply(
    lambda x: candidato_prim_afiliacion[x['Candidato']] if x['Archivo']=='2021-presidente-primarias-ingresos-candidatos.xlsx' else x['Afiliacion'],
    axis=1
)
df['Afiliacion'] = df['Afiliacion'].replace(afiliacion_replace)

In [159]:
df['Afiliacion'].value_counts(dropna=False)

Afiliado                     324949
Independiente Asociado       130370
Independiente No Asociado     28755
NaN                            4354
Name: Afiliacion, dtype: int64

# Partido, RUTPartido y DVPartido

In [160]:
def fix_rut(x):
  if x==x:
    x=str(x).replace('.0','').split('-')[0]
    if re.fullmatch(r'\d{6,8}',x):
      return x+'-'+chile_rut.verification_digit(x)
    else:
      return np.nan
  else:
    return np.nan

# updates dataframe's rows with the same rut to have the same name
def same_rut_same_name(df, rut_col, name_col, update_dict={}, exclude=[], only_include=[]):
  dict_rut_name = df[~df[name_col].isna() & ~df[name_col].isna()][[rut_col,name_col]].groupby(rut_col).agg(lambda x: max(x, key=len)).to_dict()[name_col] # keeping longest name
  dict_rut_name.update(update_dict)
  if exclude:
    dict_rut_name = {k: v for k,v in dict_rut_name.items() if k not in exclude}
  if only_include:
    dict_rut_name = {k: v for k,v in dict_rut_name.items() if k in only_include}
  df[name_col] = df.apply(lambda x: dict_rut_name[x[rut_col]] if (x[rut_col]==x[rut_col] and x[rut_col] in dict_rut_name) else x[name_col], axis=1)
  return df

# updates dataframe's rows with the same name to have the same rut
def same_name_same_rut(df, rut_col, name_col, update_dict={}, exclude=[], only_include=[]):
  dict_name_rut = df[~df[name_col].isna() & ~df[rut_col].isna()][[name_col,rut_col]].groupby(name_col).max().to_dict()[rut_col] # keeping highest rut
  dict_name_rut.update(update_dict)
  if exclude:
    dict_name_rut = {k: v for k,v in dict_name_rut.items() if k not in exclude}
  if only_include:
    dict_name_rut = {k: v for k,v in dict_name_rut.items() if k in only_include}
  df[rut_col] = df.apply(lambda x: dict_name_rut[x[name_col]] if (x[name_col]==x[name_col] and x[name_col] in dict_name_rut) else x[rut_col], axis=1)
  return df


In [161]:
# RUT correction
df['RUTPartido'] = df['RUTPartido'].map(fix_rut)
df = df.drop(columns='DVPartido')

In [162]:
# party renaming
rename_party = {
    'Partido Evolucion Politica': 'Evolucion Politica',

    'Partido Radical De Chile': 'Partido Radical',
    'Partido Radical Socialdemocrata': 'Partido Radical',

    'Independiente': np.nan,
    'Independientes': np.nan,

    'Partido Ciudadanos': 'Ciudadanos',

    'Partido Ecologista Verde Del Norte': 'Partido Ecologista Verde',
    'Partido Ecologista': 'Partido Ecologista Verde',

    'Partido De Izquierda Ciudadana De Chile': 'Partido Izquierda Ciudadana De Chile',
    'Partido De Izquierda Ciudadana De Chile -Muni 2012 Y Parl 2013': 'Partido Izquierda Ciudadana De Chile',

    'Partido Progresista De Chile': 'Partido Progresista',

    'Partido De Los Trabajadores Revolucionarios': 'Partido De Trabajadores Revolucionarios',

    'Partido Igualdad': 'Igualdad'

}
df['Partido'].replace(to_replace=rename_party, inplace=True)

In [163]:
df = same_rut_same_name(df, 'RUTPartido', 'Partido')
df = same_name_same_rut(df, 'RUTPartido', 'Partido')

In [164]:
df[['Partido','RUTPartido']].value_counts(dropna=False)

Partido                                                          RUTPartido
NaN                                                              NaN           132461
Union Democrata Independiente                                    71552600-K     52965
Renovacion Nacional                                              71395000-9     47386
Partido Democrata Cristiano                                      71468400-0     43665
Partido Por La Democracia                                        71455400-K     35788
Partido Socialista De Chile                                      71706400-3     34999
Partido Comunista De Chile                                       71701800-1     27348
Partido Radical                                                  72551500-6     16266
Convergencia Social                                              53325947-2     15682
Partido Republicano De Chile                                     65193238-6     15184
Evolucion Politica                                              

In [165]:
pr = df[['Partido','RUTPartido']].dropna()
rut_party_dict = pd.Series(pr['Partido'].values,index=pr['RUTPartido']).to_dict()
party_rut_dict = pd.Series(pr['RUTPartido'].values,index=pr['Partido']).to_dict()

In [166]:
rut_party_dict.keys()

dict_keys(['71468400-0', '71701800-1', '71395000-9', '71706400-3', '72551500-6', '71552600-K', '71447800-1', '71455400-K', '65037533-5', '65166343-1', '65171967-4', '65119848-8', '65176888-8', '65108245-5', '65102519-2', '65107248-4', '65114322-5', '65114239-3', '65123086-1', '65194007-9', '65112171-K', '53325352-0', '65144125-0', '65165842-K', '65111743-7', '65172582-8', '65092861-K', '53325947-2', '65167417-4', '65171766-3', '65193238-6', '77150977-0', '53335163-8', '65188975-8', '65202859-4', '53335692-3'])

# Candidato, RUTCandidato and DVCandidato

In [167]:
# correcting ruts
df['RUTCandidato'] = df['RUTCandidato'].map(fix_rut)
df = df.drop(columns='DVCandidato')

In [168]:
# setting same names for people with same ruts, prioritizing post-2016 records
dict_rut_candidato_2016 = df[~df['Candidato'].isna() & ~df['Candidato'].isna() & (df['AnioEleccion'].astype(int)>2016)][['RUTCandidato','Candidato']].groupby('RUTCandidato').agg(lambda x: max(x, key=len)).to_dict()['Candidato'] # keeping longest name
df = same_rut_same_name(df, 'RUTCandidato', 'Candidato', update_dict=dict_rut_candidato_2016)

In [169]:
#candidatos_many_ruts = list(df[['Candidato', 'RUTCandidato']].dropna().drop_duplicates().groupby('Candidato').filter(lambda x: len(x) > 1)['Candidato'].unique())
#df = same_name_same_rut(df, 'RUTCandidato', 'Candidato', exclude=candidatos_many_ruts)
df['RUTCandidato'] = df.apply(lambda x: party_rut_dict[x['Candidato']] if (x['RUTCandidato']!=x['RUTCandidato'] and x['Candidato'] in party_rut_dict) else x['RUTCandidato'], axis=1)

In [170]:
df[['Candidato', 'RUTCandidato']].sample(5)

,Candidato,RUTCandidato
308449,Wanda Paulina Gonzalez Figueroa,9426733-1
264598,Erick Marco Antonio Conoman Garay,16371970-3
316930,Rosa Maita Querquezana,10124383-4
346528,Roberto Alexis Santa Cruz Gonzalez,12662532-4
355161,Pablo Alejandro Pereira Pereira,13287280-5


In [171]:
cr = df[['Candidato','RUTCandidato']].dropna()
rut_candidato_dict = pd.Series(cr['Candidato'].values,index=cr['RUTCandidato']).to_dict()

In [172]:
# removing parties from candidates
df.loc[df['RUTCandidato'].isin(rut_party_dict.keys()), ['RUTCandidato', 'Candidato']] = [np.nan, np.nan]

In [173]:
df.loc[df['Candidato']=='Gabriel Boric Font',['RUTCandidato']] = '16163631-2'
df.loc[df['Candidato']=='Daniel Jadue Jadue',['RUTCandidato']] = '9400544-2'

# Aportante, RUTAportante and DVAportante

In [174]:
pd.set_option('display.max_rows', None)

In [175]:
# correcting ruts
df['RUTAportante'] = df['RUTAportante'].map(fix_rut)
df = df.drop(columns='DVAportante')

In [176]:
# setting same names for people with same ruts, prioritizing post-2016 records and including party names
dict_rut_aportante_2016 = df[~df['Aportante'].isna() & ~df['Aportante'].isna() & (df['AnioEleccion'].astype(int)>2016)][['RUTAportante','Aportante']].groupby('RUTAportante').agg(lambda x: max(x, key=len)).to_dict()['Aportante'] # keeping longest name
dict_rut_aportante_2016.update(rut_party_dict)
df = same_rut_same_name(df, 'RUTAportante', 'Aportante', update_dict=dict_rut_aportante_2016)

In [177]:
#aportantes_many_ruts = list(df[['Aportante', 'RUTAportante']].dropna().drop_duplicates().groupby('Aportante').filter(lambda x: len(x) > 1)['Aportante'].unique())
#df = same_name_same_rut(df, 'RUTAportante', 'Aportante', exclude=aportantes_many_ruts)
df['RUTAportante'] = df.apply(lambda x: party_rut_dict[x['Aportante']] if (x['RUTAportante']!=x['RUTAportante'] and x['Aportante'] in party_rut_dict) else x['RUTAportante'], axis=1)

In [178]:
df.loc[df['Aportante']=='Campana Presidencial2018 Sebastian Pinera Echenique',['Aportante']] = 'Sebastian Pinera Echenique'

In [179]:
df[['Aportante', 'RUTAportante']].dropna().sample(5)

,Aportante,RUTAportante
445166,Neil Castillo Lillo,10687824-2
277367,Formulario 107,77183141-9
123146,Carmen Rosaavilesaraya,11617365-4
239701,Paola Araya,13641621-9
478326,Rodolfo Francisco Manriquez Ortega,13652170-5


# Fecha

In [180]:
# date format transformation
ddSmmSyy = r'\d\d/\d\d/\d\d'
ddSmmSyyyy = r'\d\d/\d\d/\d\d\d\d'
ddHmmHyy = r'\d\d-\d\d-\d\d'
ddHmmHyyyy = r'\d\d-\d\d-\d\d\d\d'
yyyyHmmHdd = r'\d\d\d\d-\d\d-\d\d'
yyyyHmmHdd_time = r'\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d'

def date_trasformer(a_date):
  if re.fullmatch(ddHmmHyy, a_date) or re.fullmatch(ddSmmSyy, a_date):
    return '20'+a_date[6:8]+'-'+a_date[3:5]+'-'+a_date[0:2]
  elif re.fullmatch(ddSmmSyyyy, a_date) or re.fullmatch(ddHmmHyyyy, a_date):
    return a_date[6:10]+'-'+a_date[3:5]+'-'+a_date[0:2]
  elif re.fullmatch(yyyyHmmHdd, a_date) or re.fullmatch(yyyyHmmHdd_time, a_date):
    return a_date[0:4]+'-'+a_date[5:7]+'-'+a_date[8:10]
  else:
    return a_date

# turns everything to YYYY-MM-DD (ISO 8601)
df['Fecha'] = df['Fecha'].map(lambda x: date_trasformer(x) if x==x else x)

In [181]:
df[df['Fecha']=='1004-10-31']

,Archivo,RUTAportante,Aportante,RUTCandidato,Candidato,Afiliacion,RUTPartido,Partido,Pacto,Subpacto,...,Comuna,Region,Distrito,Circunscripcion,CircunscripcionProvincial,CircunscripcionSenatorial,TerritorioElectoral,TipoBeneficiario,AnioEleccion,TipoEleccion
26766,2004-alcalde-consejal-ingresos-candidatos.xls,11564481-5,Claudio Esteban Sepulveda Sagal,11564481-5,Claudio Esteban Sepulveda,Independiente No Asociado,NaN,NaN,Independientes,Independientes,...,Colbun,NaN,NaN,NaN,NaN,NaN,Comuna,Candidato,2004,Regular


# Tipo Aporte

In [183]:
tipo_cuenta = [
    'Anticipo Financiamiento Fiscal', # fiscal
    'Aporte Anonimo', # anonimo
    'Aporte Con Publicidad', # con publicidad, puede cambiar
    'Aporte Estimado En Especies (Mantencion Y Colaciones)', # depende
    'Aporte Estimado Por Propaganda En Espacio Privado', # depende
    'Aporte Estimado Por Propaganda En Medios De Comunicacion Digital', # depende
    'Aporte Estimado Por Propaganda En Radioemisoras', # depende
    'Aporte Estimado Por Propaganda Escrita', # depende
    'Aporte Estimado Por Servicios Profesionales De Un Tercero', # depende
    'Aporte Estimado Por Uso De Propiedad De Un Tercero', # depende
    'Aporte Estimado Por Uso De Un Vehiculo De Un Tercero', # depende
    'Aporte Patrimonial', # depende
    'Aporte Personal Del Candidato', # propio
    'Aporte Personal Del Candidato (Vehiculo, Propiedades Entre Otros)', # propio
    'Aporte Por Trabajo Voluntario', # depende
    'Aporte Por Usufructo', # depende
    'Aporte Propio', # propio
    'Aporte Propio En Efectivo Del Candidato', # propio
    'Aporte Propio En Especies O Servicios', # propio
    'Aporte Propio Por Imputacion De Reembolso A Candidatos', # propio
    'Aporte Propio Por Imputacion De Reembolso A Instituciones Financieras', # propio
    'Aporte Publico', # publico, puede cambiar
    'Aporte Publico Partido Politico', # partido
    'Aporte Reservado', # reservado
    'Aporte Trabajo Voluntario', # depende
    'Aportes Anonimos', # anonimo
    'Aportes Con Publicidad', # con publicidad, puede cambiar
    'Aportes De Partido Politico', # partido
    'Aportes De Personas Naturales', # depende
    'Aportes En Dinero De Partidos Politicos', # partido
    'Aportes En Dinero Recibido Por Candidato De Un Partido Politico', # partido
    'Aportes En Especies O Servicios De Partidos Politicos', # partido
    'Aportes En Especies O Servicios De Terceros', # depende
    'Aportes En Especies Recibido Por Candidato De Un Partido Politico', # partido
    'Aportes Menores Sin Publicidad', # sin publicidad
    'Aportes Partido Politico', # partido
    'Aportes Partidos Politicos', # partido
    'Aportes Privados De Caracter Publico', # depende
    'Aportes Privados De Caracter Publico De Partidos Politicos', # partido
    'Aportes Propios De Partido Politico', # propio
    'Aportes Reservado', # reservado
    'Aportes Sin Publicidad', # sin publicidad
    'Cesion De Derecho A Reembolso', # depende
    'Comodato', # depende
    'Credito Institucion Financiera', # banco
    'Creditos Con Instituciones Financieras', # banco
    'Creditos Con Mandato', # banco
    'Donaciones', # depende
    'Financiamiento A Candidatos Independientes', # fiscal
    'Financiamiento A Partidos Politicos', # fiscal
    'Imputacion De Gasto Electoral De Primarias', # fiscal
    'Imputacion De Gasto Presunto De Partidos Politicos', # fiscal
    'Imputacion Gasto Presunto', # fiscal
    'Imputacion Gasto Primarias', # fiscal
    'Imputacion Saldo De Financiamiento Primarias', # fiscal
    'Intereses Credito Institucion Financiera', # banco
    'Intereses Devengados Por Creditos Con Instituciones Financieras', # banco
    'Intereses Por Credito Con Institucion Financiera', # banco
    'Mutuos', # depende
    'No Definido', # depende
    'Otros Actos O Contratos A Titulo Gratuito'# depende,
    'Reembolso De Gastos Electorales', # fiscal
    'Reembolso Facturas Cedidas Partido Politico', # fiscal
    'Reembolso Solicitado Candidato O Partido', # fiscal
    'Reembolso Solicitado Por Cesion De Derecho A Partido Politico', # fiscal
    'Reembolsos Facturas Endosadas', # fiscal
    'Reembolsos Solicitado Por Endosos', # fiscal
    'Usufructo'# depende
]

In [184]:
#? 'Cesion De Derecho A Reembolso', 'No Definido',

anonimo = [ # se mantiene
    'Aporte Anonimo',
    'Aportes Anonimos',
]

publico = [ # varia segun aportante y beneficiario
    'Aporte Publico'
]

reservado = [ # se mantiene
    'Aporte Reservado',
    'Aportes Reservado',
]

sin_publicidad = [ # se mantiene
    'Aportes Menores Sin Publicidad',
    'Aportes Sin Publicidad',
]

con_publicidad = [ # varia segun aportante y beneficiario
    'Aporte Con Publicidad',
    'Aportes Con Publicidad',
]

fiscal = [ # del estado, se mantiene
    'Anticipo Financiamiento Fiscal',
    'Financiamiento A Candidatos Independientes',
    'Financiamiento A Partidos Politicos',

    'Imputacion De Gasto Electoral De Primarias',
    'Imputacion De Gasto Presunto De Partidos Politicos',
    'Imputacion Gasto Presunto',
    'Imputacion Gasto Primarias',
    'Imputacion Saldo De Financiamiento Primarias',

    'Reembolso De Gastos Electorales',
    'Reembolso Facturas Cedidas Partido Politico',
    'Reembolso Solicitado Candidato O Partido',
    'Reembolso Solicitado Por Cesion De Derecho A Partido Politico',
    'Reembolsos Facturas Endosadas',
    'Reembolsos Solicitado Por Endosos',
]

banco = [ # de una institucion financiera, se mantiene
    'Credito Institucion Financiera',
    'Creditos Con Instituciones Financieras',
    'Creditos Con Mandato',
    'Intereses Credito Institucion Financiera',
    'Intereses Devengados Por Creditos Con Instituciones Financieras',
    'Intereses Por Credito Con Institucion Financiera',
]

propio = [ # propio del candidato o del partido a si mismo, se mantiene
    'Aporte Personal Del Candidato',
    'Aporte Personal Del Candidato (Vehiculo, Propiedades Entre Otros)',
    'Aporte Propio',
    'Aporte Propio En Efectivo Del Candidato',
    'Aporte Propio En Especies O Servicios',
    'Aporte Propio Por Imputacion De Reembolso A Candidatos',
    'Aporte Propio Por Imputacion De Reembolso A Instituciones Financieras',
    'Aportes Propios De Partido Politico',
    'Aporte Patrimonial',
]

partido = [ # de un partido no a si mismo, se mantiene
    'Aporte Publico Partido Politico',
    'Aportes De Partido Politico',
    'Aportes En Dinero De Partidos Politicos',
    'Aportes En Dinero Recibido Por Candidato De Un Partido Politico',
    'Aportes En Especies O Servicios De Partidos Politicos',
    'Aportes En Especies Recibido Por Candidato De Un Partido Politico',
    'Aportes Partido Politico',
    'Aportes Partidos Politicos',
    'Aportes Privados De Caracter Publico De Partidos Politicos',
    'Reembolso Facturas Cedidas Partido Politico',
]

depende = [ # depende de quien es el aportante, aportes estimados
    'Aporte Estimado En Especies (Mantencion Y Colaciones)',
    'Aporte Estimado Por Propaganda En Espacio Privado',
    'Aporte Estimado Por Propaganda En Medios De Comunicacion Digital',
    'Aporte Estimado Por Propaganda En Radioemisoras',
    'Aporte Estimado Por Propaganda Escrita',
    'Aporte Estimado Por Servicios Profesionales De Un Tercero',
    'Aporte Estimado Por Uso De Propiedad De Un Tercero',
    'Aporte Estimado Por Uso De Un Vehiculo De Un Tercero',
    'Aporte Por Trabajo Voluntario',
    'Aporte Por Usufructo',
    'Aporte Trabajo Voluntario',
    'Aportes De Personas Naturales',
    'Aportes En Especies O Servicios De Terceros',
    'Aportes Privados De Caracter Publico',
    'Comodato',
    'Donaciones',
    'Mutuos',
    'No Definido',
    'Otros Actos O Contratos A Titulo Gratuito',
    'Usufructo',
    'Cesion De Derecho A Reembolso'
]

state_inst = {
    'Servicio Electoral': '60504000-4',
    'Tesoreria General De La Republica':'60805000-0'
}

banks = {
    'Banco Itau Chile':'76645030-K',
    'Banco De Chile':'97004000-5',
    'Banco De Credito E Inversiones':'97006000-6',
    'Banco Bilbao Vizcaya Argentaria Chile':'97032000-8',
    'Banco Santander':'97036000-K',
    'Banco Del Desarollo':'97051000-1',
    'Banco Security':'97053000-2',
    'Banco Consorcio':'99500410-0',
}

se_mantienen = anonimo+reservado+sin_publicidad+fiscal+banco

def type_by_desc(x):
  if x['DescripcionTipoCuenta'] in anonimo:
    return 'Anonimo'
  if x['DescripcionTipoCuenta'] in publico:
    return 'Publico'
  if x['DescripcionTipoCuenta'] in reservado:
    return 'Reservado'
  if x['DescripcionTipoCuenta'] in sin_publicidad:
    return 'Sin Publicidad'
  if x['DescripcionTipoCuenta'] in con_publicidad:
    return 'Con Publicidad'
  if x['DescripcionTipoCuenta'] in propio:
    return 'Propio'
  if x['DescripcionTipoCuenta'] in partido:
    return 'Partido'
  if x['DescripcionTipoCuenta'] in fiscal:
    return 'Fiscal'
  if x['DescripcionTipoCuenta'] in banco:
    return 'Banco'
  else:
    return x['TipoAporte']

def type_by_agents(x):
  if (x['TipoBeneficiario']=='Candidato' and x['RUTCandidato']==x['RUTAportante']) or (x['TipoBeneficiario']=='Partido' and x['RUTPartido']==x['RUTAportante']):
    return 'Propio'
  if x['RUTAportante'] in rut_party_dict.keys() and x['RUTAportante']!=x['RUTPartido']:
      return 'Partido'
  elif x['RUTAportante'] in state_inst.values():
    return 'Fiscal'
  elif x['RUTAportante'] in banks.values():
    return 'Banco'
  else:
    return x['TipoAporte']

def type_by_elim(x):
  if x['DescripcionTipoCuenta'] in depende:
    if int(x['AnioEleccion'])<2016:
      return 'Publico' if x['RUTAportante']==x['RUTAportante'] else 'No Identificable'
    else:
      return 'Con Publicidad' if x['RUTAportante']==x['RUTAportante'] else 'No Identificable'
  else:
    return x['TipoAporte']

df['TipoAporte'] = np.nan
print(len(df[~df['TipoAporte'].isna()]))

df['TipoAporte'] = df.apply(type_by_desc, axis=1)
print(len(df[~df['TipoAporte'].isna()]))

df['TipoAporte'] = df.apply(lambda x: type_by_agents(x) if not x['DescripcionTipoCuenta'] in se_mantienen else x['TipoAporte'], axis=1)
print(len(df[~df['TipoAporte'].isna()]))

df['TipoAporte'] = df.apply(lambda x: type_by_elim(x) if x['TipoAporte']!=x['TipoAporte'] else x['TipoAporte'], axis=1)
print(len(df[~df['TipoAporte'].isna()]))

0
295665
298012
488428


In [185]:
df['TipoAporte'].value_counts(dropna=False)

Con Publicidad      144793
No Identificable     84893
Propio               65493
Fiscal               62846
Partido              49627
Sin Publicidad       33341
Anonimo              19403
Publico              17483
Reservado             8946
Banco                 1603
Name: TipoAporte, dtype: int64

# Exporting

In [186]:
# column Re-ordering
df = df[[
    'Aportante',
    'RUTAportante',

    'Candidato',
    'RUTCandidato',
    'Afiliacion',

    'Partido',
    'RUTPartido',
    'Pacto',
    'Subpacto',

    'Monto',

    'AnioEleccion',
    'Eleccion',
    'TipoEleccion',

    'TerritorioElectoral',
    'Region',
    'CircunscripcionSenatorial',
    'Distrito',
    'CircunscripcionProvincial',
    'Comuna',

    'Fecha',
    'TipoAporte',
    'TipoBeneficiario',
    'TipoCuenta',
    'DescripcionTipoCuenta',
    'EstadoCuenta',
    'NumeroDocumento',
    'TipoDocumento',
    'DescripcionTipoDocumento',
    'GlosaDocumento',

    'Archivo'
]]

In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488428 entries, 0 to 488431
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Aportante                  421494 non-null  object 
 1   RUTAportante               286430 non-null  object 
 2   Candidato                  484345 non-null  object 
 3   RUTCandidato               482798 non-null  object 
 4   Afiliacion                 484074 non-null  object 
 5   Partido                    355967 non-null  object 
 6   RUTPartido                 344018 non-null  object 
 7   Pacto                      225156 non-null  object 
 8   Subpacto                   147134 non-null  object 
 9   Monto                      480126 non-null  float64
 10  AnioEleccion               488428 non-null  object 
 11  Eleccion                   488428 non-null  object 
 12  TipoEleccion               488428 non-null  object 
 13  TerritorioElectoral        48

In [188]:
df.nunique()

Aportante                    138463
RUTAportante                 142788
Candidato                     40161
RUTCandidato                  40469
Afiliacion                        3
Partido                         137
RUTPartido                       36
Pacto                            53
Subpacto                         52
Monto                         65536
AnioEleccion                     10
Eleccion                          8
TipoEleccion                      3
TerritorioElectoral               6
Region                           16
CircunscripcionSenatorial        19
Distrito                         60
CircunscripcionProvincial        68
Comuna                          348
Fecha                          2353
TipoAporte                       10
TipoBeneficiario                  2
TipoCuenta                       45
DescripcionTipoCuenta            68
EstadoCuenta                      2
NumeroDocumento              148251
TipoDocumento                    27
DescripcionTipoDocumento    

In [189]:
# exporting resulting data to csv
df.to_csv(work_dir + 'data/aportes.csv', index=False)

# Aportante anonimization with md5 encription (not used)

In [ ]:
# applying md5 to ruts (except to parties and candidates)
def aux_fun_md5(x):
  if x == x:
    if (x in dict_candidate_rut.values()) or (x in dict_party_rut.values()):
      return x
    else:
      return hashlib.md5(str(x).encode()).hexdigest()
  else:
    return x

raw_data['RUTAportante'] = raw_data['RUTAportante'].apply(aux_fun_md5)

In [ ]:
# cheking number of unique ruts after md5 to verify absence of hash collsions
len(raw_data['RUTAportante'].unique())

In [ ]:
# cheking values after transformations
raw_data['RUTAportante'].value_counts()